Setting AWS credentials by defining environment variables using the `os` module. It assigns the AWS access key and secret key, allowing authenticated access to AWS services.

Provide your own KEY ID and ACCESS KEY

In [1]:
import os
os.environ['AWS_ACCESS_KEY_ID'] = ""
os.environ['AWS_SECRET_ACCESS_KEY'] = ""

Installing the AWS CLI on a Linux system, configures it with AWS credentials, and prepares to interact with AWS services. It uses a bash shell script to download, unzip, install, and configure the AWS CLI.

In [2]:
# Download the AWS and Ego4D CLIs, then download the annotations locally
%%bash

# Set up the AWS CLI
curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
unzip -o awscliv2.zip >/dev/null
sudo ./aws/install >/dev/null 2>&1
aws configure set aws_access_key_id "$AWS_ACCESS_KEY_ID" && aws configure set aws_secret_access_key "$AWS_SECRET_ACCESS_KEY"
rm "awscliv2.zip"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 58.0M  100 58.0M    0     0  79.1M      0 --:--:-- --:--:-- --:--:-- 79.3M


installs the `ego4d` Python package using pip

In [3]:
!pip install ego4d

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.5/94.5 kB 1.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 114.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 9.4 MB/s eta 0:00:00
  Created wheel for ego4d: filename=ego4d-1.7.3-py3-none-any.whl size=118251 sha256=75beba368db5694eec7482fabdc3fa98731831baa26e31b4ef9d253ee420de79
  Stored in directory: /root/.cache/pip/wheels/65/a8/89/a6187e3bc9a85e81899ab8d5ddc2011c9954d3b6cb84d47e03
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31529 sha256=c999fe

uses the `ego4d` CLI to download the specified annotations and NLQ benchmark data (version v1) into the `/content/ego4d_data/` directory in Colab.

In [4]:
!ego4d --output_directory="/content/ego4d_data/" --datasets annotations --benchmarks nlq --version v1 -y

Datasets to download: {'annotations'}
Download Path: /content/ego4d_data/v1
Ego4D Metadata: /content/ego4d_data/ego4d.json
Checking requested datasets and versions...
Created download directory for version 'v1' of dataset: 'annotations' at: /content/ego4d_data/v1/annotations
Benchmarks specified but ignored without a benchmarks field in manifest.
Retrieving object metadata from S3...
100% 31/31 [00:00<00:00, 1442.00object/s]
Checking if latest file versions are already downloaded...
  3% 1/31 [00:00<00:03,  7.86file/s]WARNING:urllib3.connectionpool:Connection pool is full, discarding connection: ego4d-consortium-sharing.s3.us-west-1.amazonaws.com. Connection pool size: 10
100% 31/31 [00:00<00:00, 46.72file/s]
No existing videos to filter.
100% 2.50G/2.51G [00:14<00:00, 229MiB/s]Checking file integrity...
100% 2.51G/2.51G [00:14<00:00, 184MiB/s]


 mounts Google Drive in Colab and extracts a `.tar.gz` file containing pre-extracted features from the Ego4D dataset. The EgoVLP features, extracted from Ego4D, are used for visual learning tasks and are placed into the `/content/ego4d_data/v1/egovlp_fp16` directory.
 **make sure to place it in cited directory for running.**

In [5]:
from google.colab import drive
import shutil
import tarfile
import os

drive.mount('/content/drive')

# Copy the file from Google Drive
file_path = '/content/drive/My Drive/Colab Notebooks/egovlp_fp16.tar.gz'

tar = tarfile.open(file_path, "r")
tar.extractall(path="/content/ego4d_data/v1/egovlp_fp16")
tar.close()

Mounted at /content/drive


Checking annotations located in the proper location

In [6]:
!ls /content/ego4d_data/v1/annotations | grep nlq

nlq_test_unannotated.json
nlq_train.json
nlq_val.json


Check all videos are located in the proper location




In [7]:
!ls /content/ego4d_data/v1/egovlp_fp16 | wc -l

9611


Cloning the repository git that contains VSLNet and VSLBase models

In [8]:
%%bash
git clone https://github.com/xpuria/episodic-memory
cd episodic-memory/NLQ/VSLNet

Cloning into 'episodic-memory'...


Createing a shell script file (vars.sh) with environment variable definitions for a project. It sets up variables for dataset paths, feature directories, and model checkpoints, and changes the working directory to /episodic-memory/NLQ/VSLNet.

In [13]:
with open("vars.sh", "w") as out_f:
  out_f.write("""
export NAME=egovlp_fp16
export TASK_NAME=nlq_official_v1_$NAME
export BASE_DIR=data/dataset/nlq_official_v1_$NAME
export FEATURE_BASE_DIR=data/features/nlq_official_v1_$NAME/
export FEATURE_DIR=$FEATURE_BASE_DIR/video_features
export MODEL_BASE_DIR=/content/nlq_official_v1/checkpoints/

cd /episodic-memory/NLQ/VSLNet
"""
  )

Sourcing the vars.sh script to set environment variables, prints the FEATURE_BASE_DIR path, creates the directory if it doesn’t exist, and creates a symbolic link from /content/ego4d_data/v1/egovlp_fp16 to FEATURE_DIR.

In [14]:
%%bash

source vars.sh

echo $FEATURE_BASE_DIR
mkdir -p $FEATURE_BASE_DIR
ln -s /content/ego4d_data/v1/egovlp_fp16 $FEATURE_DIR

data/features/nlq_official_v1_egovlp_fp16/


vars.sh: line 9: cd: /episodic-memory/NLQ/VSLNet: No such file or directory


Sourcing the vars.sh script to set environment variables and then installs several Python packages, including `nltk`, `submitit`, `torch`, and others. The `%%capture` magic suppresses the output of the command

In [15]:
%%bash
%%capture

source vars.sh
pip install nltk submitit torch torchaudio torchvision tqdm transformers tensorboard Pillow terminaltables

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.7/74.7 kB 2.0 MB/s eta 0:00:00


bash: line 1: fg: no job control
vars.sh: line 9: cd: /episodic-memory/NLQ/VSLNet: No such file or directory


Sourcing vars.sh, then runs a Python script to prepare the Ego4D dataset by specifying paths for input annotations, video features, and output directories. It processes and saves the dataset in the defined locations.

In [16]:
%%bash

source vars.sh

python utils/prepare_ego4d_dataset.py \
    --input_train_split /content/ego4d_data/v1/annotations/nlq_train.json \
    --input_val_split /content/ego4d_data/v1/annotations/nlq_val.json \
    --input_test_split /content/ego4d_data/v1/annotations/nlq_test_unannotated.json \
    --video_feature_read_path /content/ego4d_data/v1/egovlp_fp16 \
    --clip_feature_save_path $FEATURE_BASE_DIR/official \
    --output_save_path $BASE_DIR

Reading [train]: /content/ego4d_data/v1/annotations/nlq_train.json
# train: 11291
Writing [train]: data/dataset/nlq_official_v1_egovlp_fp16/train.json
Reading [val]: /content/ego4d_data/v1/annotations/nlq_val.json
# val: 3874
Writing [val]: data/dataset/nlq_official_v1_egovlp_fp16/val.json
Reading [test]: /content/ego4d_data/v1/annotations/nlq_test_unannotated.json
# test: 4004
Writing [test]: data/dataset/nlq_official_v1_egovlp_fp16/test.json


vars.sh: line 9: cd: /episodic-memory/NLQ/VSLNet: No such file or directory
Extracting features:   0%|          | 0/1659 [00:00<?, ?it/s]/content/episodic-memory/NLQ/VSLNet/utils/prepare_ego4d_dataset.py:113: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Plea

 Extracting the contents of a GloVe word embeddings zip file from Google Drive into a specified directory. It creates the target directory if it doesn’t exist and lists the files in the directory after extraction. make sure to located GloVe file in the cited directory. [downlowdable via [link](https://nlp.stanford.edu/projects/glove/)]

In [17]:
import zipfile
import os

# Path to the uploaded zip file
glove_path = '/content/drive/MyDrive/glove.840B.300d.zip'

# Directory where you want to extract the contents
features_dir = '/content/episodic-memory/NLQ/VSLNet/data/features'


os.makedirs(features_dir, exist_ok=True)


with zipfile.ZipFile(glove_path, 'r') as zip_ref:
    zip_ref.extractall(features_dir)

os.listdir(features_dir)

['nlq_official_v1_egovlp_fp16', 'glove.840B.300d.txt']

In [18]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Running VSLNet Model by using Glove Encoder With 32 `BATCH_SIZE`, 10 `NUM_EPOCHS`, 0.0015 `INIT_LR`.

In [ ]:
%%bash

source vars.sh

# machine parameters
export DATALOADER_WORKERS=1
export NUM_WORKERS=2
export VAL_JSON_PATH="/content/ego4d_data/v1/annotations/nlq_val.json"

# hyper parameters
export BATCH_SIZE=32
export DIM=128
export NUM_EPOCH=10
export MAX_POS_LEN=128
export INIT_LR=0.0015

export TB_LOG_NAME="${NAME}_bs${BATCH_SIZE}_dim${DIM}_epoch${NUM_EPOCH}_ilr${INIT_LR}"

python main.py \
    --task $TASK_NAME \
    --predictor embedding \
    --dim $DIM \
    --mode train \
    --video_feature_dim 256 \
    --max_pos_len $MAX_POS_LEN \
    --init_lr $INIT_LR \
    --epochs $NUM_EPOCH \
    --batch_size $BATCH_SIZE \
    --fv official \
    --num_workers $NUM_WORKERS \
    --data_loader_workers $DATALOADER_WORKERS \
    --model_dir $MODEL_BASE_DIR/$NAME \
    --eval_gt_json $VAL_JSON_PATH \
    --tb_log_freq 5 \
    --remove_empty_queries_from train

set +x

Running with Namespace(save_dir='datasets', task='nlq_official_v1_egovlp_fp16', eval_gt_json='/content/ego4d_data/v1/annotations/nlq_val.json', fv='official', max_pos_len=128, num_workers=2, data_loader_workers=1, word_size=None, char_size=None, word_dim=300, video_feature_dim=256, char_dim=50, dim=128, highlight_lambda=5.0, num_heads=8, drop_rate=0.2, predictor='embedding', gpu_idx='0', seed=12345, mode='train', epochs=10, batch_size=32, num_train_steps=None, init_lr=0.0015, clip_norm=1.0, warmup_proportion=0.0, extend=0.1, period=100, text_agnostic=False, video_agnostic=False, model_dir='/content/nlq_official_v1/checkpoints//egovlp_fp16', model_name='vslnet', suffix=None, log_to_tensorboard=None, tb_log_dir='./runs', tb_log_freq=5, slurm=False, slurm_wait=False, slurm_partition='pixar', slurm_constraint='volta', slurm_gpus=1, slurm_cpus=10, slurm_timeout_min=720, slurm_log_folder='slurm_log', remove_empty_queries_from=['train'])
Generating data for dataloader
train: skipped = 977, re

vars.sh: line 9: cd: /episodic-memory/NLQ/VSLNet: No such file or directory
2024-09-11 14:03:47.293232: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-11 14:03:47.587651: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-11 14:03:47.670034: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-11 14:03:48.162795: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appro

Running VSLNet Model by using Glove Encoder With 32 `BATCH_SIZE`, 10 `NUM_EPOCHS`, 0.0025 `INIT_LR`.

In [ ]:
%%bash

source vars.sh

# machine parameters
export DATALOADER_WORKERS=1
export NUM_WORKERS=2
export VAL_JSON_PATH="/content/ego4d_data/v1/annotations/nlq_val.json"

# hyper parameters
export BATCH_SIZE=32
export DIM=128
export NUM_EPOCH=10
export MAX_POS_LEN=128
export INIT_LR=0.0025

export TB_LOG_NAME="${NAME}_bs${BATCH_SIZE}_dim${DIM}_epoch${NUM_EPOCH}_ilr${INIT_LR}"

python main.py \
    --task $TASK_NAME \
    --predictor embedding \
    --dim $DIM \
    --mode train \
    --video_feature_dim 256 \
    --max_pos_len $MAX_POS_LEN \
    --init_lr $INIT_LR \
    --epochs $NUM_EPOCH \
    --batch_size $BATCH_SIZE \
    --fv official \
    --num_workers $NUM_WORKERS \
    --data_loader_workers $DATALOADER_WORKERS \
    --model_dir $MODEL_BASE_DIR/$NAME \
    --eval_gt_json $VAL_JSON_PATH \
    --log_to_tensorboard $TB_LOG_NAME \
    --tb_log_freq 5 \
    --remove_empty_queries_from train

Running with Namespace(save_dir='datasets', task='nlq_official_v1_egovlp_fp16', eval_gt_json='/content/ego4d_data/v1/annotations/nlq_val.json', fv='official', max_pos_len=128, num_workers=2, data_loader_workers=1, word_size=None, char_size=None, word_dim=300, video_feature_dim=256, char_dim=50, dim=128, highlight_lambda=5.0, num_heads=8, drop_rate=0.2, predictor='embedding', gpu_idx='0', seed=12345, mode='train', epochs=10, batch_size=32, num_train_steps=None, init_lr=0.0025, clip_norm=1.0, warmup_proportion=0.0, extend=0.1, period=100, text_agnostic=False, video_agnostic=False, model_dir='/content/nlq_official_v1/checkpoints//egovlp_fp16', model_name='vslnet', suffix=None, log_to_tensorboard='egovlp_fp16_bs32_dim128_epoch10_ilr0.0025', tb_log_dir='./runs', tb_log_freq=5, slurm=False, slurm_wait=False, slurm_partition='pixar', slurm_constraint='volta', slurm_gpus=1, slurm_cpus=10, slurm_timeout_min=720, slurm_log_folder='slurm_log', remove_empty_queries_from=['train'])
Generating data 

vars.sh: line 9: cd: /episodic-memory/NLQ/VSLNet: No such file or directory
load video features: 100%|██████████| 1659/1659 [00:26<00:00, 63.64it/s]
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
evaluate val:   0%|          | 0/122 [00:00<?, ?it/s]/content/episodic-memory/NLQ/VSLNet/model/layers.py:562: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_indices // width

Epoch  10 /  10

Running VSLNet Model by using Glove Encoder With 64 `BATCH_SIZE`, 20 `NUM_EPOCHS`, 0.0025 `INIT_LR`.

In [ ]:
%%bash

source vars.sh

# machine parameters
export DATALOADER_WORKERS=1
export NUM_WORKERS=2
export VAL_JSON_PATH="/content/ego4d_data/v1/annotations/nlq_val.json"

# hyper parameters
export BATCH_SIZE=64
export DIM=128
export NUM_EPOCH=20
export MAX_POS_LEN=128
export INIT_LR=0.0025

export TB_LOG_NAME="${NAME}_bs${BATCH_SIZE}_dim${DIM}_epoch${NUM_EPOCH}_ilr${INIT_LR}"

python main.py \
    --task $TASK_NAME \
    --predictor embedding \
    --dim $DIM \
    --mode train \
    --video_feature_dim 256 \
    --max_pos_len $MAX_POS_LEN \
    --init_lr $INIT_LR \
    --epochs $NUM_EPOCH \
    --batch_size $BATCH_SIZE \
    --fv official \
    --num_workers $NUM_WORKERS \
    --data_loader_workers $DATALOADER_WORKERS \
    --model_dir $MODEL_BASE_DIR/$NAME \
    --eval_gt_json $VAL_JSON_PATH \
    --log_to_tensorboard $TB_LOG_NAME \
    --tb_log_freq 5 \
    --remove_empty_queries_from train

Running with Namespace(save_dir='datasets', task='nlq_official_v1_egovlp_fp16', eval_gt_json='/content/ego4d_data/v1/annotations/nlq_val.json', fv='official', max_pos_len=128, num_workers=2, data_loader_workers=1, word_size=None, char_size=None, word_dim=300, video_feature_dim=256, char_dim=50, dim=128, highlight_lambda=5.0, num_heads=8, drop_rate=0.2, predictor='embedding', gpu_idx='0', seed=12345, mode='train', epochs=20, batch_size=64, num_train_steps=None, init_lr=0.0025, clip_norm=1.0, warmup_proportion=0.0, extend=0.1, period=100, text_agnostic=False, video_agnostic=False, model_dir='/content/nlq_official_v1/checkpoints//egovlp_fp16', model_name='vslnet', suffix=None, log_to_tensorboard='egovlp_fp16_bs64_dim128_epoch20_ilr0.0025', tb_log_dir='./runs', tb_log_freq=5, slurm=False, slurm_wait=False, slurm_partition='pixar', slurm_constraint='volta', slurm_gpus=1, slurm_cpus=10, slurm_timeout_min=720, slurm_log_folder='slurm_log', remove_empty_queries_from=['train'])
Loading data fro

vars.sh: line 9: cd: /episodic-memory/NLQ/VSLNet: No such file or directory
load video features: 100%|██████████| 1659/1659 [00:34<00:00, 47.86it/s]
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
evaluate val:   0%|          | 0/61 [00:00<?, ?it/s]/content/episodic-memory/NLQ/VSLNet/model/layers.py:562: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_indices // width

Epoch  20 /  20:

Running VSLNet Model by using Glove Encoder With 64 `BATCH_SIZE`, 20 `NUM_EPOCHS`, 0.0015 `INIT_LR`.

In [ ]:
%%bash

source vars.sh

# machine parameters
export DATALOADER_WORKERS=1
export NUM_WORKERS=2
export VAL_JSON_PATH="/content/ego4d_data/v1/annotations/nlq_val.json"

# hyper parameters
export BATCH_SIZE=64
export DIM=128
export NUM_EPOCH=20
export MAX_POS_LEN=128
export INIT_LR=0.0015

export TB_LOG_NAME="${NAME}_bs${BATCH_SIZE}_dim${DIM}_epoch${NUM_EPOCH}_ilr${INIT_LR}"

python main.py \
    --task $TASK_NAME \
    --predictor embedding \
    --dim $DIM \
    --mode train \
    --video_feature_dim 256 \
    --max_pos_len $MAX_POS_LEN \
    --init_lr $INIT_LR \
    --epochs $NUM_EPOCH \
    --batch_size $BATCH_SIZE \
    --fv official \
    --num_workers $NUM_WORKERS \
    --data_loader_workers $DATALOADER_WORKERS \
    --model_dir $MODEL_BASE_DIR/$NAME \
    --eval_gt_json $VAL_JSON_PATH \
    --log_to_tensorboard $TB_LOG_NAME \
    --tb_log_freq 5 \
    --remove_empty_queries_from train

Running with Namespace(save_dir='datasets', task='nlq_official_v1_egovlp_fp16', eval_gt_json='/content/ego4d_data/v1/annotations/nlq_val.json', fv='official', max_pos_len=128, num_workers=2, data_loader_workers=1, word_size=None, char_size=None, word_dim=300, video_feature_dim=256, char_dim=50, dim=128, highlight_lambda=5.0, num_heads=8, drop_rate=0.2, predictor='embedding', gpu_idx='0', seed=12345, mode='train', epochs=20, batch_size=64, num_train_steps=None, init_lr=0.0015, clip_norm=1.0, warmup_proportion=0.0, extend=0.1, period=100, text_agnostic=False, video_agnostic=False, model_dir='/content/nlq_official_v1/checkpoints//egovlp_fp16', model_name='vslnet', suffix=None, log_to_tensorboard='egovlp_fp16_bs64_dim128_epoch20_ilr0.0015', tb_log_dir='./runs', tb_log_freq=5, slurm=False, slurm_wait=False, slurm_partition='pixar', slurm_constraint='volta', slurm_gpus=1, slurm_cpus=10, slurm_timeout_min=720, slurm_log_folder='slurm_log', remove_empty_queries_from=['train'])
Loading data fro

vars.sh: line 9: cd: /episodic-memory/NLQ/VSLNet: No such file or directory
2024-09-11 14:15:08.929384: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-11 14:15:08.950347: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-11 14:15:08.956724: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-11 14:15:08.971687: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appro